In [106]:
import pandas as pd

In [107]:
df =  pd.read_csv('../Bases/Criados/base_preenchida.csv')

In [108]:
# Não tem necessidade
df = df.drop('V0031', axis=1)

In [109]:
# E006011 e E008 dropados por não trazerem muita info util
df = df.drop('E008', axis=1)
df = df.drop('E006011', axis=1)

In [ ]:
# Juntei todos os atributos que diziam valor de renda que recebiam no renda

# Ajustando os valores conforme as condições especificadas
df['F001021'] = df.apply(lambda x: x['F001021'] if x['F001011'] == 1 else 0, axis=1)
df['F007021'] = df.apply(lambda x: x['F007021'] if x['F007011'] == 1 else 0, axis=1)
df['F008021'] = df.apply(lambda x: x['F008021'] if x['F008011'] == 1 else 0, axis=1)
df['VDF00102'] = df.apply(lambda x: x['VDF00102'] if x['VDF001'] == 1 else 0, axis=1)

# Somando tudo com a renda
df['Renda'] = df['Renda'] + df['F001021'] + df['F007021'] + df['F008021'] + df['VDF00102']

In [ ]:
# Dropei os atributos usados anteriormente
df = df.drop('F001021', axis=1)
df = df.drop('F007021', axis=1)
df = df.drop('F008021', axis=1)
df = df.drop('VDF00102', axis=1)
df = df.drop('F001011', axis=1)
df = df.drop('F007011', axis=1)
df = df.drop('F008011', axis=1)
df = df.drop('VDF001', axis=1)

In [112]:
# Não são muito uteis, eu sei que medem a cobertura do plano mas ta intankavel colocar isso aq
# Titular do plano de saúde
# Nº de ordem do titular
# Plano de servidor público?

# df = df.drop('I00103', axis=1)
# df = df.drop('I001031', axis=1)
# df = df.drop('I004', axis=1)

In [113]:
# Criando o novo atributo combinado
# df['Plano_de_Saúde_Cobertura'] = df['I00102'].astype(str)  # Começa com o valor de I00102

# Combinando com I00401, I00402, I00403 apenas quando I00102 for 1
#df.loc[df['I00102'] == 1, 'Plano_de_Saúde_Cobertura'] = (
#   df['I00102'].astype(str) + ' ' + 
#    df['I00401'].astype(str) + ' ' + 
#    df['I00402'].astype(str) + ' ' +  
#    df['I00403'].astype(str)
#)

# Quando I00102 for 2, mantém apenas "2" (não combina com os outros)
#df.loc[df['I00102'] == 2, 'Plano_de_Saúde_Cobertura'] = '2'

In [114]:
# 3. Filtrar apenas as linhas que atendem às condições (opcional)
#df_filtrado = df.dropna(subset=['Plano_de_Saúde_Cobertura'])

# Mostrar as primeiras linhas para verificar
#df['Plano_de_Saúde_Cobertura']

In [115]:
#df = df.drop('I00401', axis=1)
#df = df.drop('I00402', axis=1)
#df = df.drop('I00403', axis=1)

In [116]:
# J01101 Quando ____ consultou um médico pela última vez 
# H001 Quando foi a última vez que o(a) sr(a) consultou com um(a) médico(a)?
# É a mesma coisa e não é util

df = df.drop('J01101', axis=1)
df = df.drop('H001', axis=1)

# Primeira consulta com esse médico?
# Local do atendimento
# Dificuldade para atendimento
# Todas essas tão bem inuteis

df = df.drop('H002', axis=1)
df = df.drop('H004', axis=1)
df = df.drop('H010', axis=1)

In [ ]:
# calculo do imc, mas eu provavelmente vou mudar ele de lugar, pois precisamos inputar primeiro

import numpy as np

# Calculando o IMC (peso / altura²) e convertendo altura de cm para metros
df['imc'] = df['P00103'] / ((df['P00104'] / 100) ** 2)

# Classificando em faixas
conditions = [
    (df['imc'] < 18.5),
    ((df['imc'] >= 18.5) & (df['imc'] < 25)),
    ((df['imc'] >= 25) & (df['imc'] < 30)),
    ((df['imc'] >= 30) & (df['imc'] < 35)),
    ((df['imc'] >= 35) & (df['imc'] < 40)),
    (df['imc'] >= 40)
]

labels = [
    'Abaixo do peso',
    'Peso normal',
    'Sobrepeso',
    'Obesidade Grau I',
    'Obesidade Grau II',
    'Obesidade Grau III'
]

# Criando a coluna com as faixas
df['faixa_imc'] = np.select(conditions, labels, default='Dados inválidos')

# Opcional: Arredondar o IMC para 2 casas decimais
df['imc'] = df['imc'].round(2)

In [118]:
df['imc']

0        121.95
1        117.65
2        102.04
3        120.48
4           NaN
          ...  
24346    161.29
24347    161.29
24348    158.73
24349    138.89
24350    112.36
Name: imc, Length: 24351, dtype: float64

In [119]:
df['faixa_imc']

0        Obesidade Grau III
1        Obesidade Grau III
2        Obesidade Grau III
3        Obesidade Grau III
4           Dados inválidos
                ...        
24346    Obesidade Grau III
24347    Obesidade Grau III
24348    Obesidade Grau III
24349    Obesidade Grau III
24350    Obesidade Grau III
Name: faixa_imc, Length: 24351, dtype: object

In [ ]:
# Remoção dos registros vazios do atributo Morador selecionado está apto para responder? 

# Total de registros antes de remover
total_antes = len(df)

# Remover linhas onde M00203 está vazio (NaN ou string vazia)
df = df[~df['M00203'].isna() & (df['M00203'] != '')]

# Total de registros depois de remover
total_depois = len(df)

# Exibir quantos registros foram retirados
registros_removidos = total_antes - total_depois
print(f"Registros removidos: {registros_removidos}")

Registros removidos: 3


In [121]:
# Drop nesses de altura e peso pois tem muitos faltantes
df = df.drop('P00403', axis=1)
df = df.drop('P00404', axis=1)

In [ ]:
#Peso final tinha alguns registros em branco, então eu retirei eles

# Total de registros antes de remover
total_antes = len(df)

# Remover linhas onde P00104 está vazio (NaN ou string vazia)
df = df[~df['P00104'].isna() & (df['P00104'] != '')]

# Total de registros depois de remover
total_depois = len(df)

# Exibir quantos registros foram retirados
registros_removidos = total_antes - total_depois
print(f"Registros removidos: {registros_removidos}")

# Total de registros antes de remover
total_antes = len(df)

Registros removidos: 27


In [123]:
# Criando a nova coluna com a soma das horas tv e dispositivos
df['horas_dispositivos_tv'] = df['P04501'] + df['P04502']

# Verificando se há valores nulos (opcional)
print("Registros nulos na soma:", df['horas_dispositivos_tv'].isnull().sum())

Registros nulos na soma: 0


In [ ]:
# Criando a nova coluna com consumo de verduras e frutas por semana
df['consumo_frutas_verduras'] = df['P00901'].astype(str) + ' ' + df['P018'].astype(str)

In [125]:
df['consumo_frutas_verduras']

0        6.0 3.0
1        2.0 5.0
2        1.0 3.0
3        3.0 7.0
4        6.0 6.0
          ...   
24346    5.0 7.0
24347    3.0 2.0
24348    7.0 2.0
24349    7.0 3.0
24350    6.0 3.0
Name: consumo_frutas_verduras, Length: 24321, dtype: object

In [ ]:
# Dropei os atributos usados anteriormente
df = df.drop('P00901', axis=1)
df = df.drop('P018', axis=1)

In [ ]:
# Criando a nova coluna com consumo de coisas não saudaveis por semana
df['consumo_alimentos_nsaudaveis'] = (df['P02001'].astype(str) + ' ' + 
                          df['P02002'].astype(str) + ' ' + 
                          df['P02501'].astype(str) + ' ' + 
                          df['P02602'].astype(str))

In [128]:
df['consumo_alimentos_nsaudaveis']

0        0.0 2.0 1.0 3.0
1        0.0 1.0 0.0 0.0
2        0.0 4.0 0.0 3.0
3        0.0 0.0 0.0 0.0
4        0.0 0.0 0.0 0.0
              ...       
24346    0.0 0.0 0.0 0.0
24347    2.0 1.0 7.0 0.0
24348    0.0 1.0 0.0 0.0
24349    0.0 0.0 0.0 0.0
24350    0.0 6.0 0.0 0.0
Name: consumo_alimentos_nsaudaveis, Length: 24321, dtype: object

In [ ]:
# Dropei os atributos usados anteriormente
df = df.drop('P02001', axis=1)
df = df.drop('P02002', axis=1)
df = df.drop('P02501', axis=1)
df = df.drop('P02602', axis=1)

In [130]:
# Verificar colunas com dados faltantes
colunas_com_faltantes = df.columns[df.isnull().any()].tolist()

# Mostrar apenas as colunas com dados faltantes e a quantidade de faltantes
df_faltantes = df[colunas_com_faltantes].isnull().sum()
print(df_faltantes[df_faltantes > 0])

P00103             2144
Faixa_ex_fisico      69
imc                2144
dtype: int64


In [131]:
# 4. Salvar em um novo CSV (opcional)
df.to_csv('../Bases/Criados/base_plano_cartesiano.csv', index=False)